In [15]:
import os
import sys
cwd = os.getcwd()
module_path = os.path.abspath(os.path.join('..'))
sys.path.insert(0, os.path.join(cwd, "../"))

import pandas as pd
from IPython.display import display

from src.definitions import MINIMAL_PROCESSED_DATA_PATH, \
    FEATURE_CONFIG_FILE_PATH, STUDENT_FOLDER_NAME_PREFIX, BINNED_ON_VAR_FREQ_DATA_PATH
from src.utils import student_utils
from sklearn.cluster import KMeans

available_students = student_utils.get_available_students(MINIMAL_PROCESSED_DATA_PATH)
stress_details_file_name = "stress_details.csv"
stress_details = pd.DataFrame()

def array_count(array, element):
    count = 0
    
    for e in array:
        if e == element:
            count+=1
            
    return count

def stress_0(series):
    return array_count(list(series.values), 0)

def stress_1(series):
    return array_count(list(series.values), 1)

def stress_2(series):
    return array_count(list(series.values), 2)

def stress_3(series):
    return array_count(list(series.values), 3)

def stress_4(series):
    return array_count(list(series.values), 4)

aggregation_rule = {
    'stress_level': [stress_0, stress_1, stress_2, stress_3, stress_4]
    }

In [14]:
stress_details = pd.DataFrame()

for student_id in available_students:       
    stress_labels_data_path = os.path.join(MINIMAL_PROCESSED_DATA_PATH,
                                     STUDENT_FOLDER_NAME_PREFIX + str(student_id),
                                     stress_details_file_name)
    stress_data = pd.read_csv(stress_labels_data_path, index_col=[0])
    
    
    
    stress_details = stress_details.append(stress_data, ignore_index=True)
    

stress_distribution = stress_details.groupby('student_id').stress_level.agg(aggregation_rule)
kmeans = KMeans(n_clusters=3, random_state=0).fit(stress_distribution)
stress_distribution['cluster'] = kmeans.predict(stress_distribution)
stress_distribution['svg_stress_score'] = (stress_distribution[('stress_level', 'stress_0')] * 1 + 
                                          stress_distribution[('stress_level', 'stress_1')] * 2 + 
                                          stress_distribution[('stress_level', 'stress_2')] * 3 +
                                          stress_distribution[('stress_level', 'stress_3')] * 4 + 
                                          stress_distribution[('stress_level', 'stress_4')] * 5  )


stress_distribution.columns = ['_'.join(col).strip() if 'student_id' not in col else 'student_id'
                               for col in stress_distribution.columns.values]

# display(stress_distribution)
display(stress_distribution.sum())

stress_distribution.to_csv('../data/stress_distribution.csv')


/home/abhinav/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


stress_level_stress_0     117
stress_level_stress_1     462
stress_level_stress_2    1013
stress_level_stress_3     320
stress_level_stress_4     379
cluster_                   63
svg_stress_score_        7255
dtype: int64